In [4]:
from collections import defaultdict
import gzip
import numpy as np

import gzip
from collections import defaultdict

def readGz(path):
  for l in gzip.open(path, 'rt'):
    yield eval(l)

def readCSV(path):
  f = gzip.open(path, 'rt')
  f.readline()
  for l in f:
    yield l.strip().split(',')




#find most populular set ++++++++++++++++++++++++++++++++++++++++++++++
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
  bookCount[book] += 1
  totalRead += 1
    
mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0

def findPopularSet(thred:float):

    mostPopularSet = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        mostPopularSet.add(i)
        if count > totalRead/100 * thred: return mostPopularSet




#+++++++++++++++++++++++++++++++
#Jaccard 
allRating = []

userRatings = defaultdict(list)
bookUserInter = defaultdict(list)

bookIds = set()

# ratingPerItem = defaultdict(list)
all_rating_per_user = defaultdict(set)
all_rating_per_item = defaultdict(set)
all_items_per_user = defaultdict(list)
all_entries = []

interactions = []
itemIDs = defaultdict(int)
userIDs = defaultdict(int)

iId = 0
uId = 0

for user,book,r in readCSV("train_Interactions.csv.gz"):
  r = int(r)
  allRating.append(r)
  userRatings[user].append(r)
  bookUserInter[user].append(book)
  all_rating_per_item[book].add((user,r))
  all_rating_per_user[user].add((book,r))
  all_entries.append(np.array([user,book,r]))
  all_items_per_user[user].append(book)
  interactions.append([user,book,r])

  if(user not in userIDs):
     userIDs[user] = uId
     uId += 1
  
  if(book not in itemIDs):
     itemIDs[book] = iId
     iId += 1
     
  



# itemsPerUser = defaultdict(set)
# usersPerItem = defaultdict(set)



def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

# ratingPerUser = defaultdict(list)


# for u,i in allRating:
#     r = d['hours_transformed']
#     hoursPerUser[u].append(r)
#     hoursPerItem[g].append((u,r))
#     itemsPerUser[u].add(g)
#     usersPerItem[g].add(u)


#predict just based on jaccard sim
def predict_sim(u, i, rating_per_user_tr, rating_per_item_tr):
    maxSim = 0
    users_rating = set(rating_per_item_tr[i])
    for g2,_ in rating_per_user_tr[u]:
        sim = Jaccard(users_rating,set(rating_per_item_tr[g2]))
        if sim > maxSim:
            maxSim = sim

    return maxSim

pop_set = findPopularSet(58)
def predict_pop_N_sim(u, i, rpu, rpi):
   if(i in pop_set):
      return 1
   else:
      return predict_sim(u, i, rpu, rpi)
   



In [5]:

#________________________________________________________________________________\\
#BPR tensorflow
import tensorflow as tf
import random

items = list(itemIDs.keys())

class BPRbatch(tf.keras.Model):
    def __init__(self, K, lamb):
        super(BPRbatch, self).__init__()
        # Initialize variables
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        # Regularization coefficient
        self.lamb = lamb

    # Prediction for a single instance
    def predict(self, u, i):
        p = self.betaI[i] + tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.nn.l2_loss(self.betaI) +\
                            tf.nn.l2_loss(self.gammaU) +\
                            tf.nn.l2_loss(self.gammaI))
    
    def score(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        x_ui = beta_i + tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return x_ui

    def call(self, sampleU, sampleI, sampleJ):
        x_ui = self.score(sampleU, sampleI)
        x_uj = self.score(sampleU, sampleJ)
        return -tf.reduce_mean(tf.math.log(tf.math.sigmoid(x_ui - x_uj)))
    
optimizer = tf.keras.optimizers.Adam(0.1)
modelBPR = BPRbatch(5, 0.00001)

def trainingStepBPR(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleJ = [], [], []
        for _ in range(Nsamples):
            u,i,_ = random.choice(interactions) # positive sample
            j = random.choice(items) # negative sample
            while j in all_items_per_user[u]:
                j = random.choice(items)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleJ.append(itemIDs[j])

        loss = model(sampleU,sampleI,sampleJ)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

for i in range(100):
    obj = trainingStepBPR(modelBPR, interactions)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

u,i,_ = interactions[0]
# In this case just a score (that can be used for ranking), rather than a prediction of a rating


modelBPR.predict(userIDs[u], itemIDs[i]).numpy()

iteration 10, objective = 0.47010317
iteration 20, objective = 0.43735763
iteration 30, objective = 0.43920848
iteration 40, objective = 0.43076465
iteration 50, objective = 0.42868483
iteration 60, objective = 0.42117327
iteration 70, objective = 0.42514157
iteration 80, objective = 0.41768944
iteration 90, objective = 0.42485812
iteration 100, objective = 0.42137393


2.9626071

In [7]:
def predict_tensor_flow(u, i):
    return modelBPR.predict(userIDs[u], itemIDs[i]).numpy()

users_item_test_set = defaultdict(list)

for l in open("./pairs_Read.csv"):

    if l.startswith("userID"):

        continue
    u,i = l.strip().split(',')
    
    users_item_test_set[u].append(i)

countZero = 0
countOne = 0

test_set_predict = defaultdict(int)
for u in users_item_test_set:
    game_list = users_item_test_set[u]
    scores = []
    for j in game_list:
        # pred = predict_sim(u, j, all_rating_per_user, all_rating_per_item)
        # pred = predict_pop_N_sim(u, j, all_rating_per_user, all_rating_per_item)
        pred = predict_tensor_flow(u,j)

        scores.append(pred)
        if(pred == 0):
            countZero += 1
        else:
            countOne += 1
    
    scores = np.array(scores)

    sorted_indices = np.argsort(scores)

    mid_index = len(game_list) / 2

    # print(scores[sorted_indices])
    game_list = np.array(game_list)
    game_list = game_list[sorted_indices]
    print(scores[sorted_indices])
    # for ke in range()
    
    for j in range(len(game_list)):
        if(j >= mid_index):
            test_set_predict[(u,game_list[j])] = 1
        else:
            test_set_predict[(u,game_list[j])] = 0

predictions = open("./predictions_Read.csv", 'w')

for l in open("./pairs_Read.csv"):

    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
   
    pred =  test_set_predict[(u,g)]
    
    
    _ = predictions.write(u + ',' + g + ',' + str(pred) + '\n')

predictions.close()



print("count zero: ", countZero)
print("count one: ", countOne)

[0.02100506 0.90414333]
[-1.1509277   0.47410005  0.86429644  3.4782753 ]
[0.32159707 0.40091294 0.7478812  1.5583571 ]
[-1.049709    0.05244557  0.5372275   2.3217564 ]
[-0.8889649  -0.58693177 -0.29732075  1.2262173 ]
[0.8112509 1.3909574]
[-0.6437079  1.4003758]
[1.6663892 2.0366724]
[-0.1093512  1.3952208]
[1.1460774 2.167007 ]
[9.021163e-04 1.055851e+00]
[-0.10598066  2.475329  ]
[-0.6564285  1.2385736]
[-0.5746287  2.8942354]
[0.8716618 3.037139 ]
[-1.1988262  -0.87497616 -0.60293114 -0.41891614  1.1375647   1.9445695 ]
[-0.9363732  -0.4466057   0.01921687  0.07393497  0.8336585   1.802455
  2.2272694   3.6874895 ]
[0.66123486 0.79362774]
[0.37363893 1.980813  ]
[-0.16608399  2.0684133 ]
[-0.49147442 -0.01704961  0.16180073  1.0161643 ]
[-0.33921814  1.236035    1.2973738   2.8214376 ]
[-0.03671658  0.45618963  0.6815336   2.477685  ]
[-0.24904859  2.571692  ]
[-0.4635627 -0.4461934]
[-1.1954783  -0.9308046  -0.92395735  1.9466021 ]
[-0.55917144  0.27994478  2.0609348   2.3877935